# Archives endpoint

## Instantiate the client and endpoint

In [ ]:
from esios import ESIOSClient
client = ESIOSClient()
endpoint = client.endpoint(name='archives')

## List archives ids

In [ ]:
df_indicators = endpoint.list()
df_indicators

https://api.esios.ree.es/archives


,name,id,horizon,archive_type,download,date,date_times,publication_date,taxonomy_terms,vocabularies
0,REE_BalancingEnerBids,181,QM,csv,"{'name': 'REE_BalancingEnerBids', 'url': '/arc...",{'date': '2024-05-25T23:59:59.000+02:00'},[],[2024-05-25],"[{'id_taxonomy_term': 156, 'name': 'Mercados y...","[{'id_vocabulary': 20, 'name': 'Universo'}, {'..."
1,Resultado_Subasta_Mensual_POR,159,M,xls,"{'name': 'Resultado_Subasta_Mensual_POR', 'url...",{'date': '2024-05-25T23:59:59.000+02:00'},[],[2024-04-24],"[{'id_taxonomy_term': 156, 'name': 'Mercados y...","[{'id_vocabulary': 20, 'name': 'Universo'}, {'..."
...,...,...,...,...,...,...,...,...,...,...
24,totalp48,29,H,xml,"{'name': 'totalp48', 'url': '/archives/29/down...",{'date': '2024-05-25T23:59:59.000+02:00'},[2024-05-25],[2024-05-25],"[{'id_taxonomy_term': 155, 'name': 'Generación...","[{'id_vocabulary': 20, 'name': 'Universo'}, {'..."
25,totalrp48prec,27,H,xml,"{'name': 'totalrp48prec', 'url': '/archives/27...",{'date': '2024-05-25T23:59:59.000+02:00'},[2024-05-25],[2024-05-25],"[{'id_taxonomy_term': 156, 'name': 'Mercados y...","[{'id_vocabulary': 20, 'name': 'Universo'}, {'..."


https://www.esios.ree.es/es/descargas

## Select archive

- I90DIA: 34
- https://api.esios.ree.es/archives/34/download?date_type=publicacion&end_date=2024-04-21T23%3A59%3A59%2B00%3A00&locale=es&start_date=2024-04-01T00%3A00%3A00%2B00%3A00

In [ ]:
archive = endpoint.select(id=34)

https://api.esios.ree.es/archives/34


## Configure query

In [ ]:
archive.configure(data_type='real', start='2023-01-01', end='2024-05-01')

https://api.esios.ree.es/archives/34


## See metadata

In [ ]:
archive.metadata

{'archive': {'name': 'I90DIA',
  'description': 'Información de detalle por Unidad de Programación (I90DIA)',
  'updated_at': '2015-01-29 09:20:49.0',
  'id': 34,
  'horizon': 'D',
  'archive_type': 'zip',
  'download': {'name': 'I90DIA',
   'url': '/archives/34/download?date_type=datos&end_date=2024-05-01T23%3A59%3A59%2B00%3A00&locale=es&start_date=2023-01-01T00%3A00%3A00%2B00%3A00'},
  'date': {'date_type': 'datos',
   'start_date': '2023-01-01 00:00:00.000+0100',
   'end_date': '2024-05-01 23:59:59.000+0200'},
  'date_times': ['2023-01-01', '2024-03-13'],
  'publication_date': ['2023-04-25', '2024-05-25'],
  'taxonomy_terms': [{'id_taxonomy_term': 155,
    'name': 'Generación y consumo',
    'vocabulary_id': 20},
   {'id_taxonomy_term': 156,
    'name': 'Mercados y precios',
    'vocabulary_id': 20},
   {'id_taxonomy_term': 165, 'name': 'Descargas', 'vocabulary_id': 21},
   {'id_taxonomy_term': 168, 'name': 'General', 'vocabulary_id': 22}],
  'vocabularies': [{'id_vocabulary': 20, '

## Download and extract data files

In [ ]:
archive.download_and_extract(output_dir='data')

## Data preprocessing

In [5]:
import os
import pandas as pd
    
path = 'data/I3DIA/I3DIA_20230101/I3DIA_20230101.xls'

df = pd.read_excel(path, skiprows=2, sheet_name='I3DIA03', index_col=[0,1,2,3]).loc[:,1:]

date = os.path.basename(path).split('_')[1].split('.')[0]
dates = pd.date_range(date, periods=len(df.columns), freq='15min')
df.columns = dates

df = df.fillna(0).melt(ignore_index=False).reset_index().rename(columns={'variable': 'datetime', 'value': 'energy'})
df = df.sort_values(['datetime', 'Concepto']).reset_index(drop=True)

df.columns = ['redespacho', 'sentido', 'concepto', 'restriccion', 'fecha', 'energia']

df.set_index(['fecha', 'redespacho', 'sentido', 'concepto', 'restriccion'], inplace=True)
df = df.reset_index().set_index('fecha')
df

,redespacho,sentido,concepto,restriccion,energia
fecha,,,,,
2023-01-01 00:00:00,UPOPVPV,Subir,Carbón,RTT,0.000
2023-01-01 00:00:00,UPOPVPV,Subir,Carbón,RTD,0.000
...,...,...,...,...,...
2023-01-01 23:45:00,ECO,Bajar,Solar térmica,NaN,0.000
2023-01-01 23:45:00,ECO,Bajar,Turbinación bombeo,NaN,-266.475


## Export to file

In [6]:
df.to_parquet('data/I3DIA/I3DIA03_2023.parquet')